<a href="https://colab.research.google.com/github/snpsuen/Deep_Learning_Data/blob/main/script/Pytorch_MLP_Binclass_Datasetloader_AT_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split, default_collate
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from numpy import vstack

In [2]:
# Read data, convert to NumPy arrays
path = 'https://raw.githubusercontent.com/snpsuen/Deep_Learning_Data/refs/heads/main/dataset/sonar.csv'
data = pd.read_csv(path)
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

print("X shape = ", X.shape)
n_insamples = X.shape[0]
n_inputs = X.shape[1]
print("Number of input samples = ", n_insamples)
print("Number of input features = ", n_inputs)

X shape =  (208, 60)
Number of input samples =  208
Number of input features =  60


In [3]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y)
y = encoder.transform(y).reshape(-1, 1)

print("y shape = ", y.shape)
n_outsamples = y.shape[0]
n_outputs = y.shape[1]
print("Number of output samples = ", n_outsamples)
print("Number of output targets = ", n_outputs)


y shape =  (208, 1)
Number of output samples =  208
Number of output targets =  1


In [4]:
# define sonar dataset
class SonarDataset(Dataset):
    def __init__(self, X, y):
        # convert into PyTorch tensors and remember them
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        # this should return the size of the dataset
        return len(self.X)

    def __getitem__(self, idx):
        # this should return one sample from the dataset
        features = self.X[idx]
        target = self.y[idx]
        return features, target

In [5]:
# set up DataLoaders for data set
dataset = SonarDataset(X, y)
trainset, testset = random_split(dataset, [0.7, 0.3])
trainloader = DataLoader(trainset, shuffle=True, batch_size=16)
testloader = DataLoader(testset, shuffle=True, batch_size=16)

In [6]:
# create model
model = nn.Sequential(
    nn.Linear(n_inputs, 60),
    nn.ReLU(),
    nn.Linear(60, 30),
    nn.ReLU(),
    nn.Linear(30, n_outputs),
    nn.Sigmoid()
)

In [7]:
# Train the model
n_epochs = 200
loss_fn = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)
model.train()
for epoch in range(n_epochs):
    for X_batch, y_batch in trainloader:
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [8]:
# create one test tensor from the testset
X_test, y_test = default_collate(testset)
print("Type of X_test: ", type(X_test))
print("Type of y_test: ", type(y_test))
print("X_test.shape = ", X_test.shape)
print("y_test.shape = ", y_test.shape)

model.eval()
y_pred = model(X_test)
acc = (y_pred.round() == y_test).float().mean()
acc = float(acc)
print("Model accuracy: %.2f%%" % (acc*100))

Type of X_test:  <class 'torch.Tensor'>
Type of y_test:  <class 'torch.Tensor'>
X_test.shape =  torch.Size([62, 60])
y_test.shape =  torch.Size([62, 1])
Model accuracy: 82.26%


In [9]:
# evaluate the model
predictions, actuals = list(), list()
model.eval()
for i, (X_batch, y_batch) in enumerate(testloader):
  print("Iteration i: ", i)
  print("Testloader X_batch.shape = ", X_batch.shape)
  print("Testloader y_batch.shape = ", y_batch.shape)

  # evaluate the model on the test set
  yhat = model(X_batch)
  yhat = yhat.detach().numpy()
  yact = y_batch.numpy()
  yact = yact.reshape((len(yact), 1))

  print("yhat.shape = ", yhat.shape)
  print("yact.shape = ", yact.shape)

  # round to class values
  yhat = yhat.round()
  # store
  predictions.append(yhat)
  actuals.append(yact)

predictions, actuals = vstack(predictions), vstack(actuals)
# calculate accuracy
acc = accuracy_score(actuals, predictions)
print('Accuracy = %.3f' % acc)

Iteration i:  0
Testloader X_batch.shape =  torch.Size([16, 60])
Testloader y_batch.shape =  torch.Size([16, 1])
yhat.shape =  (16, 1)
yact.shape =  (16, 1)
Iteration i:  1
Testloader X_batch.shape =  torch.Size([16, 60])
Testloader y_batch.shape =  torch.Size([16, 1])
yhat.shape =  (16, 1)
yact.shape =  (16, 1)
Iteration i:  2
Testloader X_batch.shape =  torch.Size([16, 60])
Testloader y_batch.shape =  torch.Size([16, 1])
yhat.shape =  (16, 1)
yact.shape =  (16, 1)
Iteration i:  3
Testloader X_batch.shape =  torch.Size([14, 60])
Testloader y_batch.shape =  torch.Size([14, 1])
yhat.shape =  (14, 1)
yact.shape =  (14, 1)
Accuracy = 0.823
